## Importing libraries

In [5]:
import requests
import json
import time

## Reading credentials from JSON

In [2]:
with open('assembly_ai_credentials.json', 'r') as f:
    credentials = json.load(f)
    
API_KEY = credentials['assemly_ai_api_key']
print('API_KEY', API_KEY[:2])

API_KEY 6b


## Upload and transcribe a file

In [3]:
upload_endpoint = "https://api.assemblyai.com/v2/upload"
transcript_endpoint = "https://api.assemblyai.com/v2/transcript"

In [4]:
def read_file(filename):
    with open(filename, 'rb') as f:
        return f.read()
    
read_file('downloads/transcribeDimo1677013500488.json')

b'{"TranscriptionJob": {"TranscriptionJobName": "transcribeDimo1677013500488", "TranscriptionJobStatus": "COMPLETED", "LanguageCode": "en-US", "MediaSampleRateHertz": 44100, "MediaFormat": "mp3", "Media": {"MediaFileUri": "s3://dimo-transcribe-hubermanlab-podcasts/Paul-Rosenberg.mp3"}, "Transcript": {"TranscriptFileUri": "https://s3.eu-central-1.amazonaws.com/aws-transcribe-eu-central-1-prod/282245150475/transcribeDimo1677013500488/083da55c-1f83-44e1-b513-a7ec65488a91/asrOutput.json?X-Amz-Security-Token=IQoJb3JpZ2luX2VjECwaDGV1LWNlbnRyYWwtMSJHMEUCIEvAUlgcfrozszUtMuoKKXwyqp7F0amCCm9TUr0QeIVXAiEAtRkNG4D0Cw08Ijk%2BA%2F5DB8Ksgo%2BqPGcqrCO4pdgKe%2F0q3wQIxf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARADGgw2MDI2NTkyODE0NDUiDOFH0z7gyURDYU4xXSqzBKxiFRgrzaBZa77AFjCTA2vP6AZtmLetZRnKhgb%2BudU8Fk6Q89%2FEzm7PQVCbowTxeu0DfCpvDMN5gtjP7IZCAH0UaGe3bPfWG3sn9fhwDJyEwr9fNiE8yDqsNX7J8Qprl2i455H1CH4YOtgtNMJYnCGJJfG7%2BsbNpLNbWdei0pb5FsHtK3UpGXXDngR1erIN%2BM3O8wDgRo9C%2B23UPSXfrkf6DnE7R8N2%2B8GQSKzjqTNCuyGMDX6ex7im7RoFuAu

In [7]:
header = {
	'authorization': API_KEY,
	'content-type': 'application/json'
}

def upload_file(audio_file, header):
    upload_response = requests.post(
        upload_endpoint,
        headers=header, data=read_file(audio_file)
    )
    return upload_response.json()

upload_response = upload_file('downloads/Ep. 139: Creator & Creatures Give & Receive.mp3', header)
upload_response

{'upload_url': 'https://cdn.assemblyai.com/upload/b70687ef-09f8-4074-92b0-c8ad1c9bfc64'}

In [8]:
def request_transcript(upload_url, header):
    transcript_request = {
        'audio_url': upload_url
    }
    transcript_response = requests.post(
        transcript_endpoint,
        json=transcript_request,
        headers=header
    )
    return transcript_response.json()

transcript_req_response = request_transcript(upload_response['upload_url'], header)
transcript_req_response

{'id': 'raaiyft9n6-dc49-4889-a4de-1c4e7c475217',
 'language_model': 'assemblyai_default',
 'acoustic_model': 'assemblyai_default',
 'language_code': 'en_us',
 'status': 'queued',
 'audio_url': 'https://cdn.assemblyai.com/upload/b70687ef-09f8-4074-92b0-c8ad1c9bfc64',
 'text': None,
 'words': None,
 'utterances': None,
 'confidence': None,
 'audio_duration': None,
 'punctuate': True,
 'format_text': True,
 'dual_channel': None,
 'webhook_url': None,
 'webhook_status_code': None,
 'webhook_auth': False,
 'webhook_auth_header_name': None,
 'speed_boost': False,
 'auto_highlights_result': None,
 'auto_highlights': False,
 'audio_start_from': None,
 'audio_end_at': None,
 'word_boost': [],
 'boost_param': None,
 'filter_profanity': False,
 'redact_pii': False,
 'redact_pii_audio': False,
 'redact_pii_audio_quality': None,
 'redact_pii_policies': None,
 'redact_pii_sub': None,
 'speaker_labels': False,
 'content_safety': False,
 'iab_categories': False,
 'content_safety_labels': {},
 'iab_cat

In [9]:
def get_paragraphs(polling_endpoint, header):
    paragraphs_response = requests.get(polling_endpoint + "/paragraphs", headers=header)
    paragraphs_response = paragraphs_response.json()

    paragraphs = []
    for para in paragraphs_response['paragraphs']:
        paragraphs.append(para)

    return paragraphs

def poll_transcript(trn_id, header):
    polling_endpoint = f'https://api.assemblyai.com/v2/transcript/{trn_id}'
    while True:
        polling_response = requests.get(polling_endpoint, headers=header)
        polling_response = polling_response.json()

        if polling_response['status'] == 'completed':
            return get_paragraphs(polling_endpoint, header)

        time.sleep(5)
        
paragraphs = poll_transcript(transcript_req_response['id'], header)
paragraphs

[{'text': "Here's an open and relational thought from Thomas J. Ord every theology makes assumptions about how God may be similar to or different from creatures. This is the first in a series that looks at those similarities and differences and asks about analogies between Creator and creatures. I think an open and relational theology of love ought to make positive or constructive claims about who God is and who creatures are and how they might both differ from one another and be similar. The first analogy I want to explore might say this Creator and creatures give and receive in relationship.",
  'start': 3450,
  'end': 59920,
  'confidence': 0.67713,
  'words': [{'text': "Here's",
    'start': 3450,
    'end': 3658,
    'confidence': 0.67713,
    'speaker': None},
   {'text': 'an',
    'start': 3674,
    'end': 3806,
    'confidence': 0.81841,
    'speaker': None},
   {'text': 'open',
    'start': 3828,
    'end': 4062,
    'confidence': 0.60145,
    'speaker': None},
   {'text': 'an

In [10]:
text = ''.join([p['text'] for p in paragraphs])
text

"Here's an open and relational thought from Thomas J. Ord every theology makes assumptions about how God may be similar to or different from creatures. This is the first in a series that looks at those similarities and differences and asks about analogies between Creator and creatures. I think an open and relational theology of love ought to make positive or constructive claims about who God is and who creatures are and how they might both differ from one another and be similar. The first analogy I want to explore might say this Creator and creatures give and receive in relationship.I believe both God and creatures are relational. That is, they affect others and are affected, and their love is both giving and receiving. God and creatures are passable. To use the ancient language, I don't accept classical theisms claim that God only gives and never receives. I think we should affirm that God and creatures can give and receive love in relationship with one another.In other words, both th